# Data enrichment

### <u>Content:<u>

[1) Load data sets

[2) Add week day and month to reservation data
    
[3) Estimate part of the Swiss population that is on holiday for any given date
    
[4) Add coordinates for start and destination 

[5) Get weather data
    
[6) Get capacity data for trains in 2020 according to the expected train formation


In [66]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import seaborn as sns
import scipy
from scipy.spatial.distance import cdist
import datetime
import math
import meteomatics.api as api
import datetime as dt
%matplotlib inline
rcParams['figure.figsize']=10,8

ModuleNotFoundError: No module named 'meteodata'

#### 1) Load data

In [49]:
#read the reservation data 
df = pd.read_csv('data_raw/reservation_data_2019-2021_incl_capacity.csv', 
                 parse_dates=["date"], date_parser=lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S"))         
        
print('reservations: ')
display(df.tail(1))


# read holiday data
df_schulferien = pd.read_csv('data_preprocessed/Schulferien.csv', dtype={"canton": "string", "population": "int32"})
df_schulferien['start'] = pd.to_datetime(df_schulferien['holidays_start'])
df_schulferien['end'] = pd.to_datetime(df_schulferien['holidays_end'])
df_schulferien = df_schulferien.drop(columns=["holidays_start", "holidays_end"])

print('schulferien: ')
display(df_schulferien.tail(1))

# train station coordinates data
df_coordinates = pd.read_csv('data_preprocessed/dienststellen.csv')
df_coordinates = df_coordinates[["abk_bahnhof", "lat", "lon"]]
df_coordinates = df_coordinates[df_coordinates['abk_bahnhof'].notna()]
display(df_coordinates.tail(1))

# weather data
def parse_date(row):
    date=datetime.datetime.fromisoformat('2019-11-11T00:52:43+00:00')
    return date

df_weather = pd.read_csv("data_preprocessed/weather.csv", dtype={"leisure_biking:idx": "float", "t_2m:C": "float","precip_24h:mm":"float" })
df_weather['date']= df_weather.apply(lambda row: parse_date(row), axis=1)
display(df_weather.tail(1))

# jahresformation
df_jahresformation = pd.read_csv("data_preprocessed/jahresformation.csv", dtype={"Block Bezeichnung": "string"})
display(df_jahresformation.tail(1))
df_jahresformation.dtypes

# kapazität
df_kapazität = pd.read_csv("data_preprocessed/rollmaterial-matching.csv")
display(df_kapazität.tail(1))


reservations: 


,Unnamed: 0,res_id,res_dt,date,train_nr,line,reserved,capacity,bp_from,bp_to,dep_ist,dep_soll,arr_ist,arr_soll,res_delta_ist,res_delta_soll,res_delta_valid
226735,226735,246187,2021-10-31 17:16:00,2021-10-31,1635,IC 51,1,6.0,BI,DMT,2021-10-31 19:49:00,2021-10-31 19:49:00,2021-10-31 20:17:00,2021-10-31 20:18:00,9216.0,9210,True


schulferien: 


,canton,population,start,end
161,national,7917100,2022-12-25,2022-12-26


C:\Users\tabea\miniconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,abk_bahnhof,lat,lon
47113,BRIA,47.260083,7.942674


,station_id,validdate,leisure_biking:idx,snow_depth:cm,t_2m:C,precip_24h:mm,weather_symbol_1h:idx,effective_cloud_cover:octas,date
4744,didok_8594580,2021-12-31T14:00:00Z,0.0,37.5,7.9,0.0,1,0,2019-11-11 00:52:43+00:00


C:\Users\tabea\miniconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Zug,ZNS,VP,Debicode,Zugart,Suf.,Traktion,Bhf von,Ländercode UIC,von,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
177418,96765,4,-162,1604,S,NaN,NaN,RUS,85,21:42:12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Block Bezeichnung in Jahresformation Fpl-2022,No. of hooks
36,Re460 AS4 A4(LBT) Bpm61 3B4(LBT) Bt4,10


#### 2) Week day <a name="stat"></a>

- Add a feature for weekday: 'weekday' and 'month'

In [3]:
df['weekday'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month

#### 3) Holidays <a name="stat"></a>
- Add a feature for number of people in holiday canton: n_holiday

In [4]:
# for each date, get the number of people in Switzerland who 
# are either on school holiday or national holiday..
def get_holiday_people(date):
    filtered_holidays = df_schulferien[(df_schulferien['start']<=date)&(df_schulferien['end']>=date)]
    is_national_holiday = (filtered_holidays["canton"]=="national").sum()
    if is_national_holiday:
        people = 7917100
    elif not(filtered_holidays.empty):
        filtered_holidays = filtered_holidays[filtered_holidays["canton"]!="national"]
        people = sum(filtered_holidays["population"])
    else: people=0
    return people

In [5]:
# filter df, only 2021 data
# df=df[df['date']>='2021-01-01']
df['holiday_people'] = df.apply(lambda row : get_holiday_people(row['date']), axis = 1)


KeyboardInterrupt: 

#### 4) Coordinates <a name="hr"></a>

In [6]:
# full join for start train station 
df = pd.merge(df, df_coordinates, left_on='bp_from', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_from", "lon": "lon_from"})

# full join for destination
df = pd.merge(df, df_coordinates, left_on='bp_to', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_to", "lon": "lon_to"})
display(df.head(2))

,Unnamed: 0,res_id,res_dt,date,train_nr,line,reserved,capacity,bp_from,bp_to,...,arr_soll,res_delta_ist,res_delta_soll,res_delta_valid,weekday,month,lat_from,lon_from,lat_to,lon_to
0,0,0,2019-03-29 00:00:00,2019-04-01,510,IC 5,1,NaN,ZUE,NE,...,2019-04-01 08:32:00,284667.0,284634,False,0,4,47.378177,8.540212,46.996727,6.935702
1,283,290,2019-04-06 00:00:00,2019-04-06,1528,IC 5,1,NaN,ZUE,NE,...,2019-04-06 18:01:00,59422.0,59418,False,5,4,47.378177,8.540212,46.996727,6.935702


#### 5) Weather <a name="corr"></a>

In [68]:
# summarize daily weather data
#df_weather
#df_weather['date'] = pd.to_datetime(df.date)
#df_weather = df_weather.resample('d', on='date')[['leisure_biking:idx', "t_2m:C", "precip_24h:mm"]].agg(['mean'])
#df = pd.merge(df, df_weather, left_on='date', right_on='date')
display(df_weather)

,leisure_biking:idx,t_2m:C,precip_24h:mm
,mean,mean,mean
date,,,
2019-04-01,0.000000,-3.320930,6.364651
2019-04-02,0.000000,0.306250,2.026875
2019-04-03,0.000000,-0.636000,0.945600
2019-04-04,0.000000,2.862963,6.828889
2019-04-05,0.426667,12.003333,5.759333
2019-04-06,0.565263,11.594737,3.829158
2019-04-07,0.072973,1.724324,2.414459
2019-04-08,0.388421,8.810526,2.470526


In [102]:
##import meteo data 
def get_meteodata(lat, lon, year, month, day):
    username = 'can-guru_otth'
    password = 'eyk47W6ATq'
    coordinates = [(lat, lon)]
    model =     'mix'
    startdate = dt.datetime(year=year, month=month, day=day, hour=0, minute=0, second=0)
    enddate =   startdate 
    interval = dt.timedelta(hours=0)
    format = 'csv' 

    today=dt.datetime.now()
    delta_days =(startdate-today).days
    
    # for longer forecast, use standard values for leisure biking idx + precipiation
    if delta_days>16:
        parameters = ['t_2m:C']#,'precip_24h:mm', 'leisure_biking:idx']
        df = api.query_time_series(coordinates, startdate, enddate, interval, parameters, username, password, model=model)
        # insert mean values
        df['leisure_biking:idx']= 0.39
        df['precip_24h:mm']=3.9

        
    else: 
        parameters = ['t_2m:C','precip_24h:mm', 'leisure_biking:idx']
        df = api.query_time_series(coordinates, startdate, enddate, interval, parameters, username, password, model=model)

    return df
    
# example function call
get_meteodata(47.378177, 8.540212, 2022, 3, 29)


,,,t_2m:C,precip_24h:mm,leisure_biking:idx
lat,lon,validdate,,,
47.378177,8.540212,2022-03-29 00:00:00+00:00,5.5,0.0,0.5


#### 6) Capacity <a name="corr"></a>

 - Für jede Reservation: Zugnummer im Jahresformation-Datensatz abrufen
 - Beachte: richtiges Jahr wählen, häufigste Formation
 - entsprechende Kapazität auslesen

In [ ]:
#combine data from capacity table and annual formation
list_kapazitäten=df_kapazität["Block Bezeichnung in Jahresformation Fpl-2022"].tolist()
df_jahresformation = df_jahresformation[df_jahresformation["Block Bezeichnung"].isin(list_kapazitäten)]
df_jahresformation=df_jahresformation[["Block Bezeichnung", "Zug", "Beginn Fahrplanperiode"]]
df_jahresformation = pd.merge(df_jahresformation, df_kapazität, left_on='Block Bezeichnung', right_on='Block Bezeichnung in Jahresformation Fpl-2022').drop(columns=["Block Bezeichnung in Jahresformation Fpl-2022"])
df_jahresformation.tail(2)

In [12]:
median_df=df[df["capacity"].notnull()]
median=np.median(median_df['capacity'])
print('Median Kapazität', median)

Median Kapazität 12.0


In [19]:
def fill_capacity(row):
    train=row.train_nr   
    formation = df_jahresformation[df_jahresformation["Zug"]==train]
    capacity= formation["No. of hooks"]
    return capacity

In [20]:
# for the rows where there is no capacity included
# first try to check with the jahresformation lookup table
mask = df.capacity.isnull()
df['capacity'] = df[mask].apply(fill_capacity, axis=1)

# secondly, fill the median capacity 
df=df.fillna(value={"capacity": median})

AttributeError: module 'pandas' has no attribute 'average'

In [ ]:
df.head(10)